# Mini-BMN Notebook

Expand mini-BMN hamiltonian into Pauli strings

To Do
- add comments
- check units
- how to simulate? what to simulate?
- fuzzy spheres

In [3]:
import qiskit
import numpy as np
import sympy as sp
from collections import Counter
from qiskit.circuit import Parameter, ParameterVector, ParameterExpression
from qiskit.quantum_info import Pauli, Operator, SparsePauliOp
#from symengine.lib.symengine_wrapper import Zero as spZero
import symengine.lib.symengine_wrapper as symengine_wrapper
from utils import SpecialUnitaryGroup, BMNModel
from typing import Union

In [2]:
import matplotlib
import matplotlib.pyplot as plt
from cycler import cycler

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 5.0
plt.rcParams['xtick.minor.size'] = 3.0
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rc('font', family='serif',size=14)
#matplotlib.rc('text', usetex=True)
matplotlib.rc('legend', fontsize=14)
plt.rcParams['ytick.minor.size'] = 3.0
matplotlib.rcParams.update({"axes.grid" : True,
                            "grid.alpha": 0.75,
                            "grid.linewidth": 0.5})
matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B8'])
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Blah

In [17]:
bmn = BMNModel(gauge_group_degree=2, bits_per_oscillator=1)
nu = Parameter('nu')
bmn.hamiltonian(nu)

number_qubits 9, Hilbert space dimension 512


SparsePauliOp(['IIIIIIIII', 'XIIIXIIIX', 'XIIIIXIXI', 'IXIXIIIIX', 'IXIIIXXII', 'IIXXIIIXI', 'IIXIXIXII', 'XXIXXIIII', 'XIXXIXIII', 'IXXIXXIII', 'XXIIIIXXI', 'XIXIIIXIX', 'IXXIIIIXX', 'IIIXXIXXI', 'IIIXIXXIX', 'IIIIXXIXX'],
              coeffs=[ParameterExpression(2.25*nu**2 + 4.5),
 ParameterExpression(-2.12132034355964*nu),
 ParameterExpression(2.12132034355964*nu),
 ParameterExpression(2.12132034355964*nu),
 ParameterExpression(-2.12132034355964*nu),
 ParameterExpression(-2.12132034355964*nu),
 ParameterExpression(2.12132034355964*nu), (-0.25000000000000006+0j),
 (-0.25000000000000006+0j), (-0.25000000000000006+0j),
 (-0.25000000000000006+0j), (-0.25000000000000006+0j),
 (-0.25000000000000006+0j), (-0.25000000000000006+0j),
 (-0.25000000000000006+0j), (-0.25000000000000006+0j)])

In [19]:
Hmat = bmn.hamiltonian(nu).assign_parameters({nu:1}).to_matrix(sparse=False)
Hmat

array([[6.75+0.j, 0.  +0.j, 0.  +0.j, ..., 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 6.75+0.j, 0.  +0.j, ..., 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 6.75+0.j, ..., 0.  +0.j, 0.  +0.j, 0.  +0.j],
       ...,
       [0.  +0.j, 0.  +0.j, 0.  +0.j, ..., 6.75+0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, ..., 0.  +0.j, 6.75+0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, ..., 0.  +0.j, 0.  +0.j, 6.75+0.j]])

In [24]:
x = np.linalg.eigvals(Hmat)
xR = np.real(x)
xI = np.imag(x)

xR[np.isclose(xR, 0)] = 0
xI[np.isclose(xI, 0)] = 0

assert np.array_equal(xI, np.zeros(len(xI)))

In [25]:
xR

array([15.98528137,  6.5       ,  4.5       ,  6.5       ,  6.5       ,
        6.5       ,  6.5       , 15.98528137, 15.98528137,  4.5       ,
        4.5       ,  6.5       ,  4.5       , 15.98528137, 15.98528137,
       15.98528137,  4.5       ,  4.5       ,  4.5       ,  6.5       ,
        6.5       , 15.98528137, 15.98528137, 15.98528137, 15.98528137,
        6.5       , -0.98528137, -0.98528137, -0.98528137, -0.98528137,
       -0.98528137, -0.98528137, -0.98528137, -0.98528137, -0.98528137,
       -0.98528137, -0.98528137, -0.98528137, -0.98528137, -0.98528137,
       -0.98528137, -0.98528137, -0.98528137, -0.98528137, -0.98528137,
       -0.98528137, -0.98528137, -0.98528137, -0.98528137, -0.98528137,
       -0.98528137, -0.98528137, -0.98528137, -0.98528137, -0.98528137,
       -0.98528137, -0.98528137, -0.98528137, -0.98528137, -0.98528137,
       -0.98528137, -0.98528137, -0.98528137, -0.98528137, -0.98528137,
       -0.98528137, -0.98528137, -0.98528137, -0.98528137, -0.98

In [5]:
def maximum_weight_of_sparse_pauli(operator: SparsePauliOp) -> int:
    return max([bmn.num_qubits - p.__str__().count("I") for p in operator.paulis])

In [6]:
# max pauli weight of creation operators
for matrix_idx in range(bmn.num_matrices):
    for generator_idx in range(bmn.num_matrices):
        print(maximum_weight_of_sparse_pauli(bmn.creation_operator(matrix_idx=matrix_idx, generator_idx=generator_idx)))

1
1
1
1
1
1
1
1
1
